In [1]:
import os
import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
from sklearn.model_selection import train_test_split as tts

c:\Users\Glen\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load pre-trained ViT model and feature extractor
model_name = 'google/vit-base-patch16-224-in21k'
model = ViTForImageClassification.from_pretrained(model_name, num_labels=2)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Glen\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [3]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])

# Load dataset
dataset = ImageFolder('../data2', transform=transform)

In [4]:
train_dataset, test_dataset = tts(dataset, test_size=0.2)

c:\Users\Glen\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:
len(train_dataset), len(test_dataset)

(752, 188)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Define optimizer and loss function
optimizer = Adam(model.parameters(), lr=5e-5)
criterion = CrossEntropyLoss()

# Fine-tune model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            val_loss += criterion(outputs, labels).item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / len(val_loader.dataset)
    print(f'Epoch {epoch + 1}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
            
    torch.save(model.state_dict(), os.path.join('vision-transformer-models', f'model{epoch}.pth'))

100%|██████████| 24/24 [05:35<00:00, 14.00s/it]


Epoch 1, Val Loss: 0.0075, Val Accuracy: 0.9309


100%|██████████| 24/24 [05:37<00:00, 14.08s/it]


Epoch 2, Val Loss: 0.0067, Val Accuracy: 0.9202


100%|██████████| 24/24 [08:08<00:00, 20.34s/it]


...

Epoch 8, Val Loss: 0.0061, Val Accuracy: 0.9309


100%|██████████| 24/24 [08:00<00:00, 20.03s/it]


Epoch 9, Val Loss: 0.0062, Val Accuracy: 0.9309


100%|██████████| 24/24 [10:02<00:00, 25.09s/it]


Epoch 10, Val Loss: 0.0063, Val Accuracy: 0.9362
